# Sediment production, transport and deposition in both continental and marine environments

This notebook shows a run example of the `marine_model` provided in [fastscape](https://fastscape.readthedocs.io/en/latest/).

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape

%matplotlib inline

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Import and inspect the model

In [ ]:
from fastscape.models import marine_model

In [ ]:
marine_model

In [ ]:
marine_model.visualize(show_inputs=True)

## Model setup

In [ ]:
in_ds = xs.create_setup(
    model=marine_model,
    clocks={
        'time': np.arange(0, 4e6 + 1e3, 1e3),
        'out': np.arange(0, 4e6 + 2e5, 2e5),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [101, 101],
        'grid__length': [4e5, 4e5],
        'boundary__status': ['fixed_value', 'core', 'looped', 'looped'],
        'init_topography': {
            'x_left': 0,
            'x_right': 2.5e5,
            'elevation_left': -1e3,
            'elevation_right': 0.
        },
        'uplift': {
            'x_position': 2.5e5,
            'rate_left': 0.,
            'rate_right': 3e-4
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef_bedrock': 1e-5,
            'k_coef_soil': 1e-5,
            'g_coef_bedrock': 1.,
            'g_coef_soil': 1.,
        },
        'diffusion': {
            'diffusivity_bedrock': 1e-2,
            'diffusivity_soil': 1e-2
        },
        'marine': {
            'ss_ratio_land': 0.5,
            'porosity_sand': 0.,
            'porosity_silt': 0.,
            'e_depth_sand': 1e3,
            'e_depth_silt': 1e3,
            'diffusivity_sand': 5e2,
            'diffusivity_silt': 2.5e2,
            'layer_depth': 1e2,
        },
        'strati': {
            'freeze_time': ('horizon', np.linspace(0, 4e6, 5))
        }
    },
    output_vars={
        'topography__elevation': 'out',
        'strati__elevation': 'out',
        'marine__ss_ratio_sea': 'out',
    }
)

in_ds

## Run the model

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=marine_model)

In [ ]:
out_ds

## Plot the outputs (topography and stratigraphic horizons)

In [ ]:
import hvplot.xarray
import holoviews as hv
from xshade import hillshade


ss_ratio_plot = out_ds.marine__ss_ratio_sea.hvplot.image(
    x='x', y='y', clim=(0, 1),
    width=550, height=450,
    cmap='viridis_r', groupby='out'
)

hillshade_plot = hillshade(out_ds, 'out').hvplot.image(
    x='x', y='y', cmap='gray', alpha=0.5,
    colorbar=False, hover=False, groupby='out'
)

yhorizons_plot = out_ds.strati__elevation.sel(y=2e5).hvplot.line(
    by='horizon', groupby='out', ylim=(-1000, 1000),
    width=550, legend='bottom_right',
)

hv.Layout((ss_ratio_plot * hillshade_plot) + yhorizons_plot).cols(1)